In [1]:
import pandas as pd
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from common_functions import get_teams_from_db

team_ids = [team.id for team in get_teams_from_db()]


In [2]:
def get_team_roster_by_team_id(team_id):
    try:
        roster = commonteamroster.CommonTeamRoster(team_id=team_id)
        roster_df = roster.get_data_frames()[0]
        
    except Exception as e:
        print(f"An error occurred: {e}")
        print(roster_df.get_json())
    
    return roster_df

In [21]:
rosters = [get_team_roster_by_team_id(team_id) for team_id in team_ids]


        TeamID SEASON LeagueID             PLAYER  NICKNAME  \
0   1610612737   2023       00      Jalen Johnson     Jalen   
1   1610612737   2023       00      Trent Forrest     Trent   
2   1610612737   2023       00         Seth Lundy      Seth   
3   1610612737   2023       00        Kobe Bufkin      Kobe   
4   1610612737   2023       00    Dejounte Murray  Dejounte   
5   1610612737   2023       00         Trae Young      Trae   
6   1610612737   2023       00    De'Andre Hunter  De'Andre   
7   1610612737   2023       00  Bogdan Bogdanovic    Bogdan   
8   1610612737   2023       00         AJ Griffin        AJ   
9   1610612737   2023       00       Clint Capela     Clint   
10  1610612737   2023       00     Onyeka Okongwu    Onyeka   
11  1610612737   2023       00     Mouhamed Gueye  Mouhamed   
12  1610612737   2023       00      Dylan Windler     Dylan   
13  1610612737   2023       00     Bruno Fernando     Bruno   
14  1610612737   2023       00   Garrison Mathews  Garr

In [23]:
print(type(rosters[0]))

<class 'pandas.core.frame.DataFrame'>


In [24]:
from datetime import datetime

from models import Player

players = []
for team in rosters:
    team = team.to_dict("records")
    for player in team:
        player_record = Player(
            name=player['PLAYER'],
            nickname=player.get('NICKNAME'),  # Using .get() to handle cases where the key might not exist
            player_slug=player['PLAYER_SLUG'],
            jersey_number=player['NUM'],
            position=player['POSITION'],
            height=player['HEIGHT'],
            weight=float(player['WEIGHT']) if player['WEIGHT'] else None,
            birth_date=datetime.strptime(player['BIRTH_DATE'], '%b %d, %Y') if player['BIRTH_DATE'] else None,
            age=int(player['AGE']) if player['AGE'] else None,
            experience=player['EXP'],
            school=player['SCHOOL'],
            player_id=player['PLAYER_ID'],
            how_acquired=player['HOW_ACQUIRED'],
            team_id=player['TeamID']  # Make sure the 'TeamID' matches your Team table ID
        )
        players.append(player_record)
    

In [27]:
from db_config import get_session
session = get_session()

try:
    session.add_all(players)
    session.commit()
except Exception as e:
    session.rollback()  # Rollback if there are any errors
    raise e  # Optionally re-raise the exception after handling it
finally:
    session.close()  # Always close the session